# Geocodificación de siniestros para la ciudad de Rosario

En el presente notebook se realiza el proceso de geocodificación de los siniestros ocurridos en la ciudad de Rosario, Santa Fe, Argentina. 

El procedimiento se divide en varios pasos que aseguran la limpieza de las calles, las cuales fueron ingresadas manualmente, para garantizar una geocodificación precisa. Finalmente, el proceso concluye con la propia geocodificación, utilizando Nominatim para direcciones puntuales (por ejemplo, Pasco 1550) y OpenStreetMap para intersecciones (por ejemplo, Oroño y Pellegrini).

In [7]:
import difflib
import matplotlib.pyplot as plt
import numpy as np
import osmnx as ox
import pandas as pd
import plotly.express as px
import re
import time
from geopy.geocoders import Nominatim
from shapely.geometry import LineString, MultiLineString, MultiPoint, GeometryCollection, Point
from unidecode import unidecode
import geopandas as gpd

## Limpieza de datos

Se comienza leyendo el CSV *'data_final'* y filtrando la ciudad de Rosario.

In [8]:
data = pd.read_csv("data_final.csv")
data = data[data['desc_loc']=='ROSARIO'].copy()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18316\3693281426.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data_final.csv")


Luego, se crea una nueva columna llamada *'forma_geocod'*, que contendrá la información de geocodificación. Esta puede provenir de los datos ya presentes en el CSV, de la geocodificación realizada con Nominatim para direcciones puntuales, o de la geocodificación de puntos en intersecciones obtenida con OpenStreetMap para siniestros en intersecciones.

In [9]:
data['forma_geocod'] = np.nan # columna nueva con todos nulos

# si la direccion ya estaba geocodificada --> guardo en forma_geocod la geocodificación existente
data.loc[data['posicion_XY'].notnull(), 'forma_geocod'] = data['posicion_XY']

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18316\3445348552.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['-32.942611111111,-60.651583333333' '-32.987866, -60.629612'
 '-32.975833333333,-60.727222222222' ...
 '-32.965971377348964, -60.6859731090'
 '-32.978710625552154, -60.6351299015' '    -32.96786, -60.64194']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.loc[data['posicion_XY'].notnull(), 'forma_geocod'] = data['posicion_XY']


Como sólo nos interesan los siniestros ocurridos en la ciudad, filtraremos los datos para ignorar aquellos que hayan ocurrido en rutas o autopistas.

In [10]:
# separo los accidentes en ruta y los accidentes en ciudad
accidentes_ruta = data[data['desc_ruta'].notna() | data['calle_avenida_km'].str.contains('ruta|km|kilometro|autopista|autovia', case=False, na=False)].copy()
accidentes_ciudad = data[~data.index.isin(accidentes_ruta.index)].copy()

A continuación, se define la función **'separar_en_calles'**. Esta función tiene como objetivo devolver una lista con las calles involucradas en el siniestro. Si la dirección es nula, devolverá una lista vacía. Si se trata de una dirección puntual, devolverá una lista con un único elemento. Si se trata de una intersección, devolverá una lista con dos (o más) elementos.

Además, se crean dos nuevas columnas: *'calle_lista'* y *'numero_calles'*. La primera contendrá los resultados de la función, es decir, una lista con la(s) calle(s) involucrada(s) en el siniestro. La segunda contendrá el número de elementos en dicha lista, indicando el número de calles participantes. Esta última columna será útil para filtrar los siniestros al usar Nominatim o OSM.

In [11]:
def separar_en_calles(dire):
    '''Recibe una dirección. Si es vacía, devuelve una lista vacía. Si es puntual, devuelve una lista de 
    un único elemento. Si es una intersección, devuelve una lista con cada calle participante'''
    if pd.isna(dire):
        return []
    else:
        calles = re.split(" y | entre | e | casi ", dire, flags=re.IGNORECASE)
        calles = [x for x in calles if x != '']
        return calles
    
# llamo a la función y también creo una columna 'numero_calles' que calcula cuántos elementos hay en cada lista de 'calle_lista'
accidentes_ciudad['calle_lista'] = accidentes_ciudad['calle_avenida_km'].apply(lambda x: separar_en_calles(x))  # calles lista
accidentes_ciudad['numero_calles'] = accidentes_ciudad['calle_lista'].apply(lambda x: len(x))  # nro calles 

La función previamente definida presenta dificultades con las calles que contienen un 'y' en su nombre, como en los casos de 'Guido y Spano' o 'López y Planes', ya que interpreta cada parte del nombre por separado, lo que resulta en elementos adicionales en la lista. Para abordar estos casos, se implementa la función **'aplicar_transformaciones'** para unificar los nombres.

In [12]:
def aplicar_transformaciones(dataframe, transformaciones):
    ''''Recibe un dataframe y una lista de diccionarios. Aplica transformaciones en el dataframe según cada diccionario'''
    for index, row in dataframe.iterrows():
        calle_lista = row['calle_lista']
        for i, transformacion in enumerate(transformaciones):
            if any(transformacion['condicion'] in calle.lower() for calle in calle_lista):
                calle_lista = [transformacion['nuevo_valor'] if calle.lower() == transformacion['condicion'] else calle for calle in calle_lista]
                calle_lista = [calle for calle in calle_lista if calle.lower() != transformacion['eliminar_valor']]
                if 'eliminar_valor2' in transformacion:
                    calle_lista = [calle for calle in calle_lista if calle.lower() != transformacion['eliminar_valor2']]
                dataframe.at[index, 'calle_lista'] = calle_lista

transformaciones = [
    {'condicion': 'planes', 'eliminar_valor': 'av lopez', 'eliminar_valor2': 'lopez', 'nuevo_valor': 'López y Planes'},
    {'condicion': 'spano', 'eliminar_valor': 'guido', 'nuevo_valor': 'Guido y Spano'},
    {'condicion': 'acha', 'eliminar_valor': 'pje. suriguez', 'nuevo_valor': 'Suriguez y Acha'},
    {'condicion': 'sanabria', 'eliminar_valor': 'pje. trejo', 'nuevo_valor': 'Trejo y Sanabria'}]

aplicar_transformaciones(accidentes_ciudad, transformaciones)

A diferencia de los casos anteriores, en Rosario existen las calles 'Cullen', 'Ugarte' y 'Cullen y Ugarte', lo que ha generado dificultades al definir las listas en la columna *'calle_lista'*. Es por eso que, a continuación, se abordan estos casos específicos.

La calle 'Cullen' y la calle 'Ugarte' no se cortan, por lo tanto no existe intersección entre ellas. Esto es importante aclarar porque hay casos en la base de datos de siniestros donde ambos nombres figuran sin una numeración, entonces se decidió tratar estos casos como la calle 'Cullen y Ugarte', aclarando que su geocodificación no podrá ser posible al tratarse de una dirección incompleta. 

Luego, algunos casos registrados como 'Cullen y Ugarte 1500' han sido incorrectamente separados debido a la 'y' que une ambos nombres. En estos casos, se unificará toda la dirección en un único elemento, tratándose de una dirección puntual.

Finalmente, si la lista de calles contiene tres elementos, indicando una intersección entre 'Cullen y Ugarte' y otra calle, se unificarán 'Cullen y Ugarte' como una sola entidad, representando la intersección entre las dos calles.

In [13]:
for index, row in accidentes_ciudad.iterrows():
    calle_lista = row['calle_lista']

    calle_lista_lower = [calle.lower() for calle in calle_lista]
    
    # caso 1: uno 'Cullen' y 'Ugarte' cuando son los únicos elementos (dirección incompleta)
    if len(calle_lista_lower) == 2 and 'cullen' in calle_lista_lower and 'ugarte' in calle_lista_lower:
        calle_lista = ['Cullen y Ugarte']
    
    # caso 2: uno 'Cullen' y 'Ugarte' cuando 'Cullen' está presente y 'Ugarte' tiene un número (dirección puntual)
    elif len(calle_lista_lower) == 2 and 'cullen' in calle_lista_lower:
        match = re.match(r'ugarte\s+(\d+)', calle_lista[1].lower())
        if match:
            calle_lista = ['Cullen y Ugarte ' + match.group(1)]
    
    # caso 3: uno 'Cullen' y 'Ugarte' cuando 'Cullen' y 'Ugarte' están presentes  (es una intersección entre Cullen y Ugarte + otra calle)
    elif len(calle_lista_lower) == 3 and 'cullen' in calle_lista_lower and any(calle == 'ugarte' or calle == 'ugarte ' for calle in calle_lista_lower):
        calle_lista = [calle for calle in calle_lista if calle.lower() != 'cullen' and calle.lower() != 'ugarte' and calle.lower() != 'ugarte ']
        calle_lista.insert(0, 'Cullen y Ugarte')
    
    accidentes_ciudad.at[index, 'calle_lista'] = calle_lista

Una vez tratados estos casos particulares (se verán más adelante otros, como 'Battle y Ordóñez'), es importante considerar que al geocodificar intersecciones, OpenStreetMap es mucho más exigente que Nominatim para direcciones puntuales.

Mientras que Nominatim puede geocodificar una dirección incluso si no se escribe el nombre de la calle de manera completamente precisa, OpenStreetMap requiere que la calle esté escrita de manera exacta según lo que se encuentra en su base de datos.

Por ejemplo, Nominatim puede reconocer la calle 'Peru', pero OpenStreetMap necesita que esté escrita como 'Perú'. Lo mismo sucede con 'Avellaneda': en algunos casos, OSM puede reconocerlo, pero en la mayoría de los casos será necesario escribir 'Avenida Nicolás Avellaneda' para que OpenStreetMap pueda geocodificar la dirección correctamente.

Es por esa razón que se define la función **'unificar'**, la cual recibe el nombre de una calle y, mediante el uso de expresiones regulares, lo modifica según lo propuesto en los diccionarios de reemplazos.

In [14]:
def unificar(calle, reemplazos):    
    '''Recibe el nombre de una calle y un diccionario de reemplazos, y acorde a los cambios dentro del diccionario reemplazos, devuelve la calle modificada'''
    nueva_calle = calle.lower()
    for pattern, replacement in reemplazos.items():
        nueva_calle = re.sub(pattern, replacement, nueva_calle, flags=re.IGNORECASE)
    nueva_calle = re.sub(r'\.', ' ', nueva_calle)
    nuevas_palabras = [palabra.capitalize() for palabra in nueva_calle.split() if palabra]
    nuevas_palabras = ' '.join(nuevas_palabras)

    return nuevas_palabras

Para llevar a cabo estos reemplazos, se definen dos diccionarios. El primero de ellos, llamado *'reemplazos_puntuales'*, se utilizará para los siniestros que contienen un solo elemento en la lista de la columna *'calle_lista'*, es decir, direcciones puntuales. Aunque se ha mencionado que Nominatim es generalmente más robusto en la geocodificación, hay casos en los que es necesario considerar errores de ortografía, abreviaciones o errores tipográficos.

Por otro lado, el segundo diccionario, *'reemplazos_inter'*, será utilizado para reemplazar las calles de los siniestros que ocurren en intersecciones (con dos o más elementos en *'calle_lista'*). Como se observa, este diccionario es más extenso, ya que los siniestros en intersecciones son mucho más comunes que los ocurridos en direcciones puntuales. Además, como se mencionó anteriormente, OpenStreetMap es más exigente en la geocodificación de intersecciones, por lo tanto se necesita un diccionario más amplio y detallado.

In [15]:
reemplazos_puntuales = {
        r'\b(?:av(\.|enida)?|avda(\.)?|cortada|bv(\.)?)\b': '',
        r'\bal\b': '',
        r'\bpte(\.|)?\b': 'Presidente',
        r'\bpje(\.)?\b': 'Pasaje',
        r'\bgral(\.|)?\b': 'General',
        r'\btte(\.|)?\b': 'Teniente',
        r'\b(?:pte(\.|)?\s*peron|godoy(?!\scruz\b))\b': 'Avenida Presidente Perón',
        r'\boro(\.|)?o|oro?o|boroño|oroño|oro o|oro\?o|oro�o\b': 'Nicasio Oroño',
        r'\bbs as|bs(\.|)?\s*as(\.|)?|bs\. as\.\b': 'Buenos Aires',
        r'\b(?:rou(ll)?(?:in|on|ion)?|romillon|roullon)\b': 'Alfredo Rouillón',
        r'\bpcias(\.|)?\b': 'Provincias',             
        r'\b(?:segu(i)?|busegui|bv\.?segui)\b': 'Bulevar Juan Francisco Seguí',
        r'\bov(\.|)?\b': 'Ovidio',                     
        r'\bJ(\.|)?\s*C(\.|)?\s*Casas\b': 'Juan Carlos Casas',
        r'\bctes(\.)?\b': 'Corrientes',
        r'\b(?:estados unidos|eeuu|ee(\.|)? uu(\.|)?|e(\.|)? e(\.|)? u(\.|)? u(\.|)?|ee\. uu)\b': 'Avenida 25 de Mayo',
        r'\b(?:lesgarza|lesganje)\b': 'Joaquín lejarza',
        r'\bM(\.)?\s*DE ESTRADA\b': 'Martinez de Estrada',
        r'\bb(\.)?\s*ordoñez\b': 'Avenida Battle y Ordoñez',
        r'\bP.L.Funes\b': 'Pedro Lino Funes',
        r'\b1&#186; de mayo|1 de mayo|1ero de mayo\b': 'Primero de Mayo',
        r'J(\.|)? M(\.|)? De Rosas|J(\.|)? M(\.|)? Rosas\b\b': 'Juan Manuel de Rosas',
        r'\bReg(\.|)?\s*11 De Infanteria|reg(\.|)?\s*11\b': 'Regimiento 11',
        r'\bJj(\.|)?\s*Paso|J(\.|)?\s*J(\.|)?\s*Pasos|J(\.|)?\s*J(\.|)?\s*Paso|JUAN J(\.|)?\s*PASOS\b': 'Juan José Paso',
        r'\bwheelwright|weelwrigth\b': 'Avenida Guillermo Wheelwright',
        r'\bJ(\.|)?\s*NEWERY|J(\.|)?\s*newbery\b': 'Jorge Newbery',
        r'\birigoyen|yrigoyen|hirigoyen|Hipolito Irygoyen\b': 'Avenida Hipólito Irigoyen',
        r'\bdr(\.|)?\s*rivas|dr(\.|)?\s*riva\b': 'Doctor Francisco Riva',
        r'\bcoria\b': 'Loria'
    }

In [16]:
reemplazos_inter = {
        r'\b(?:av(\.|enida)?|avda(\.)?|pasaje|cortada|cda(\.|)?|calle|pje(\.|)?|bv(\.)?|boulevard|gral(\.|)?|bis|pte(\.|)?)\b': '',
        r'\b(?:godo(?:y)?|(\w\.+\s*)+peron)|juan peron|J D peron|j d peron|peron|PTE PERON\b': 'Avenida Presidente Perón',
        r'\bav. godoy|AV GODO|AV GODOY|av. godo|godo|godoy|av. godot\b': 'Avenida Presidente Perón',
        r'\boro(\.|)?o|oro?o bis|boroño|oroño|oro o|oro\?o|oro�o\b': 'Nicasio Oroño',
        r'\bbs as|bs as|bs\. as\.|bs as\b': 'Buenos Aires',
        r'\b(?:rou(ll)?(?:in|on|ion)?|romillon|rouillon|ramillon|rovillon)\b': 'Alfredo Rouillón',
        r'\bpcias(\.|)?\b': 'Provincias',             
        r'\b(?:segu(i)?|busegui|bv\.?segui)\b': 'Bulevar Juan Francisco Seguí',
        r'\bov(\.|)?\b': 'Ovidio',                     
        r'\b1724\b': 'Hermana Paula Márquez',                  
        r'\b1715\b': 'Plácido Grela',
        r'\b1481\b': 'Seren',
        r'\b1707\b': 'Martín Fierro', 
        r'\b1750\b': 'Ana María Zeno', 
        r'\b1344\b': 'Coliqueo', 
        r'\b1829\b': 'Pierina Pasotti', 
        r'\b1809\b': 'Gaucho Rivero',
        r'\b1112\b': 'Juan José Saer',
        r'\b2105\b':'La Cumparsita', 
        r'\b1635\b': 'Suinda', 
        r'\b13101\b': 'Rubén Naranjo',
        r'\b1717\b': 'Eduardo Isern', 
        r'\b1211\b': 'Néstor Ferraza', 
        r'\b1348\b': 'Liliana Maresca', 
        r'\b1819\b': 'Aborígenes Argentinos', 
        r'\b1737\b': 'Lia Bauman', 
        r'\b1711\b':'Carlos Uriarte',
        r'\b1701\b':'Arturo Jauretche', 
        r'\b808\b': 'Lidice', 
        r'\b1333\b':'Domingo Candia', 
        r'\b1001\b':'Simon Wiesenthal', 
        r'\b2103\b': 'Caminito', 
        r'\b1422\b':'Saladillo', 
        r'\b1844\b':'Héctor Miguel Rolla', 
        r'\b1845\b': 'Alfredo Jorge Vázquez',
        r'\b1807\b':'Juan XXIII', 
        r'\b1754\b': 'Olga Cossettini', 
        r'\b1406\b': 'Pilcomayo', 
        r'\b1345\b': 'Graciela Lo Tufo', 
        r'\b1815\b': 'Avenida Mario Cisneros',
        r'\b1115\b': 'Gaboto',
        r'\bctes(\.)?\b': 'Corrientes',
        r'\bprimo\b': 'Primero',
        r'\barijon\b': 'Avenida Manuel Arijón',
        r'\bgrandol(i|u)?\b': 'Avenida Abanderado Grandoli',
        r'\b(?:estados unidos|eeuu|ee(\.|)? uu(\.|)?|e(\.|)? e(\.|)? u(\.|)? u(\.|)?|ee\. uu)\b': 'Avenida 25 de Mayo',
        r'\b(?:Sabine|travesia|Sabino|Sabin|sobia)\b': 'Avenida Travesía Albert Sabin',
        r'\b(?:San\sDiego|diego)\b': 'Bulevar San Diego',
        r'\b(?:C\.?\sCasas|Avenida\sCasas)\b': 'Avenida Casiano Casas',
        r'\b(?:lesgarza|lesganje)\b': 'Joaquín lejarza',
        r'\bnelson\b':'Pasaje Don Orione',
        r'\bcarballo|tres vias|camballo|3 vias\b':'Avenida Doctor Luis Cándido Carballo',
        r'\b(?:Estarda|M\sEstrada|J\sM\sEstrada)\b': 'Estrada', 
        r'\bwilde\b': 'Bulevar Wilde', 
        r'\b(?:cortada\sricardone|pasaje\sricardone)\b': 'Ricardone',
        r'\bprevision\b':'5 de Agosto',
        r'\bolmos|avenida de los olmos|olmos bis\b' : 'Los Olmos',
        r'\bB. ORDOÑEZ\b': 'Avenida Battle y Ordoñez',
        r'\b(?:ord[oó]ñez|ordonez|ordo�ez|ordo[nñ]ez|ordo\?ez)\b': '',
        r'\bbatle|battle|batlle|bv\.? ordonez|bv. ordoñez|av\. battle|av\. batlle|bv(\.)?\s*ordoñez\b': 'Avenida Battle y Ordoñez',
        r'\bfrancia|francia 100 Bis\b': 'Avenida Francia',
        r'\bdean funes|d. funes|d funes\b':'Deán Gregorio Funes',
        r'\bhogar|Hogar\b':'',     
        r'\bInterseccion Junin|Junin Al 100|Junin 1600|Junin 500\b' : 'Junín',
        r'\b1&#186; de mayo|1 de mayo|1ero de mayo\b': 'Primero de Mayo',
        r'J(\.|)? M(\.|)? De Rosas|J(\.|)? M(\.|)? Rosas\b\b': 'Juan Manuel de Rosas',
        r'\bReg(\.|)?\s*11 De Infanteria|reg(\.|)?\s*11\b': 'Regimiento 11',
        r'\bJ(\.|)? Cura|Jorge Cura\b': 'Avenida Jorge Cura',
        r'\bJj(\.|)?\s*Paso|J(\.|)?\s*J(\.|)?\s*Pasos|J(\.|)?\s*J(\.|)?\s*Paso 700bis\b': 'Juan José Paso',
        r'\bForest|Foret 7500|Frest 1200\b':  'Carlos Forest',
        r'\bmr(\.|)? ross|m(\.|)? ross|ross\b': 'Rodrigo M. Ross',
        r'\bwheelwright|weelwrigth\b': 'Avenida Guillermo Wheelwright',
        r'\birigoyen|yrigoyen|hirigoyen|Hipolito Irygoyen\b': 'Avenida Hipólito Irigoyen',
        r'\bacceso sur|acceso\b': 'Belgrano',
        r'\bviedma|biedma\b': 'Coronel Biedma',
        r'\bdr(\.|)?\s*rivas|dr(\.|)?\s*riva\b': 'Doctor Francisco Riva',
        r'\b[a-zA-Z]\b': ''
    }

In [17]:
# uso de la función 'reemplazos' para cada elemento de las listas de la columna 'calle_lista' si es una dirreción puntual
accidentes_ciudad.loc[accidentes_ciudad['numero_calles'] == 1, 'calle_lista'] = accidentes_ciudad[accidentes_ciudad['numero_calles'] == 1]['calle_lista'].apply(lambda lista: [unificar(calle, reemplazos_puntuales) for calle in lista])

# uso de la función 'reemplazos' para cada elemento de las listas de la columna 'calle_lista' si es una intersección
accidentes_ciudad.loc[accidentes_ciudad['numero_calles'] >= 2, 'calle_lista'] = accidentes_ciudad[accidentes_ciudad['numero_calles'] >= 2]['calle_lista'].apply(lambda lista: [unificar(calle, reemplazos_inter) for calle in lista])

Como se mencionó anteriormente, hay dos casos adicionales donde la función para separar las calles ha generado problemas: 'Battle y Ordóñez' y 'Previsión y Hogar' (actualmente '5 de Agosto'). Estos casos no se han abordado anteriormente debido a que los nombres fueron cambiados con el diccionario de reemplazos.

En el siguiente bloque de código, se itera sobre cada fila del DataFrame y, si alguno de los elementos de la lista en *'calle_lista'* coincide con los nombres de alguna de las calles ('Battle y Ordóñez' o '5 de Agosto'), se eliminan los elementos nulos o vacíos de la lista, si los hay. Además, si otro elemento de la lista comienza con un número o con 'A' o 'Al', lo que indica una dirección puntual, se unen todos los elementos de la lista en un único elemento, resultando en una dirección puntual. Por ejemplo, 'Battle y Ordóñez 1500'.

In [18]:
patron = re.compile(r'\b\d+\b') # patrón para números

for index, row in accidentes_ciudad.iterrows():
    if isinstance(row['calle_lista'], list):

        row['calle_lista'] = [calle.replace('Avenida Battle Ordoñez', 'Avenida Battle y Ordoñez') if isinstance(calle, str) else calle for calle in row['calle_lista']]
        if 'Avenida Battle y Ordoñez' in row['calle_lista'] or '5 De Agosto' in row['calle_lista']: # verifico si 'Avenida Battle y Ordoñez' o '5 de Agosto' están en la lista
            row['calle_lista'] = [calle for calle in row['calle_lista'] if pd.notnull(calle)] # filtro elementos nulos de 'calle_lista'
            
            for i, calle in enumerate(row['calle_lista']):
                if calle == '' and i > 0:
                    del row['calle_lista'][i] # elimino elementos vacíos
            
            # uno todos los elementos en un único string si cumplen las condiciones (otro elemento es un número o comienza con 'A' o 'Al')
            for i, calle in enumerate(row['calle_lista']):
                if isinstance(calle, str) and calle not in ['Avenida Battle y Ordoñez', '5 De Agosto']:
                    if patron.fullmatch(calle) or calle.startswith('A ') or calle.startswith('Al '):
                        row['calle_lista'] = [' '.join(row['calle_lista'])]
                        break

        row['calle_lista'] = [calle for calle in row['calle_lista'] if pd.notnull(calle)]
        accidentes_ciudad.at[index, 'calle_lista'] = row['calle_lista'] # actualizo el df

# calculo nuevamente el número de elementos de cada lista, actualizando 'numero_calles'
accidentes_ciudad['numero_calles'] = accidentes_ciudad['calle_lista'].apply(lambda x: len(x) if (isinstance(x, list) and len(x) > 0 and x[0].strip() != '') else 0) 

Para poder geocodificar las intersecciones, es necesario cargar los datos de la red de Rosario con el siguiente código. Este código utiliza el archivo gpkg *'municipios_santa_fe.gpkg'*, el cual contiene información geográfica sobre todos los municipios y comunas de la provincia de Santa Fe.

En caso de que se desee obtener la red de otro municipio, simplemente se debe agregar a la lista *'municipios'*, que por el momento contiene un único elemento: 'Rosario'.

In [19]:
municipios_argentina = gpd.read_file('municipios_santa_fe.gpkg')

municipios = ['Rosario'] # si se desea geocodificar otro municipio, agregarlo a la lista.

datos_municipios = {} # diccionario para almacenar los datos de cada municipio

for municipio in municipios:

    municipio_data = municipios_argentina[municipios_argentina['nam'] == municipio].copy() 
    
    # obtengo los límites del municipio
    bounds = municipio_data.geometry.bounds
    north, south, east, west = bounds['miny'].values[0], bounds['maxy'].values[0], bounds['maxx'].values[0], bounds['minx'].values[0]
    
    # descargo datos de la red de OpenStreetMap para el municipio
    G = ox.graph_from_bbox(north, south, east, west, network_type='all_private', simplify=False, retain_all=True)
    G = ox.project_graph(G)
    
    # convierto los datos de la red a GeoDataFrames
    nodes, edges = ox.utils_graph.graph_to_gdfs(G)
    
    # reproyecto los GeoDataFrames a EPSG 4326
    crs = 'EPSG:4326'
    nodes = nodes.to_crs(crs)
    edges = edges.to_crs(crs)
    
    # relleno los nombres de las calles faltantes con una cadena vacía
    edges['name'].fillna('', inplace=True)
    
    # guardo los GeoDataFrames en el diccionario de datos de municipios
    datos_municipios[municipio] = {'nodes': nodes, 'edges': edges}

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18316\922448198.py:16: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead.
  G = ox.graph_from_bbox(north, south, east, west, network_type='all_private', simplify=False, retain_all=True)


A continuación, definiremos la variable *'calles_osm_rosario'*, que contendrá una lista con todos los nombres de las calles de Rosario, según lo especificado en el archivo de municipios. Esta lista será de utilidad para la geocodificación de las intersecciones.

In [20]:
calles_osm_rosario = datos_municipios['Rosario']['edges']['name'].unique()  # lista con las calles de Rosario

Como se mencionó la importancia de tener los nombres exactos de las calles (al menos para las intersecciones), se define la función **'encontrar_similares'**. Esta función busca coincidencias entre el nombre de la calle ingresado y los nombres que figuran en la lista *'calles_osm_rosario'*, devolviendo aquellos con una probabilidad de similitud mayor o igual al 70%. 

Esta técnica será útil para aquellos casos de calles que estén mal escritas, ya sea por errores de tipeo u ortografía. Por ejemplo, 'Oronio' encontrará una coincidencia con 'Oroño'.

In [21]:
def encontrar_similares(calle, calles_osm):
    '''Recibe el nombre de una calle y devuelve aquella con mayor similitud dentro de la base de datos de OSM,
    con una probabilidad mayor o igual al 70%'''
    similares = {}
    no_match_list = []  # acá guardo los nombres sin match
    
    if calle.isdigit():
        # si el nombre de la calle es un número, crea posibles variantes con 'Pasaje' y 'Calle'
        posibles_variantes = [f'Pasaje {calle}', f'Calle {calle}']
        best_match = None
        best_ratio = 0
        
        for variante in posibles_variantes:
            closest_match = difflib.get_close_matches(variante, calles_osm, cutoff=0.7, n=1)
            if closest_match:
                match = closest_match[0]
                ratio = difflib.SequenceMatcher(None, variante, match).ratio() # calculo el ratio de similitud
                if ratio > best_ratio:  # me quedo con el mejor ratio encontrado
                    best_match = match
                    best_ratio = ratio
        
        if best_match:
            similares[calle] = best_match
        else:
            similares[calle] = calle  # si no hay match, conservo la calle original
            no_match_list.append(calle)  # agrego también a la lista de no matches
    else:
        closest_match = difflib.get_close_matches(calle, calles_osm, cutoff=0.7, n=1)
        if closest_match:
            match = closest_match[0]  
            similares[calle] = match 
        else:
            similares[calle] = calle 
            no_match_list.append(calle)
    
    return similares, no_match_list

Como interesa tratar con esta función los casos de las intersecciones, se filtran aquellas filas con un número de calles mayor o igual a 2.

In [22]:
# filtro las calles que son intersecciones, porque sino me borra los números de las direcciones puntuales
calle_listas_filtradas = accidentes_ciudad[accidentes_ciudad['numero_calles'] >= 2]['calle_lista']

# llamo la función a cada elemento de 'accidentes_ciudad['calle_lista']'
results = [encontrar_similares(calle, calles_osm_rosario) for lista in calle_listas_filtradas for calle in lista]

similares = {calle: match for similares, _ in results for calle, match in similares.items()}
no_match_list = [calle for _, no_match_list in results for calle in no_match_list]

print("Similares:")
for calle_erronea, closest_match in similares.items():
    if closest_match is not None:
        print(f"Closest match for '{calle_erronea}': '{closest_match}'")
    else:
        print(f"No match found for '{calle_erronea}'")

print("\nCalles sin match:")
for street_name in no_match_list:
    print(street_name)

Al observar detalladamente los resultados, hay calles para las cuales el algoritmo de matching asignó un valor incorrecto. Por tanto, es necesario corregirlos manualmente.

Es importante destacar que algunas calles no pueden ser geocodificadas debido a que no existen en la base de datos de OSM o no son válidas para la geocodificación, como por ejemplo 'vías del ferrocarril' o 'club nob'. Si el algoritmo de matching les asignó una coincidencia, se añadirán a la lista *'eliminar'*. Esto permitirá eliminarlas del diccionario *'similares'*, evitando asignarles un nombre incorrecto.

In [ ]:
eliminar = ['El Trebol', 'Las Palmeras', 'Larguia', 'San Martín', 'Serrano', '1740', 'Del Valle', 'Cabal', 'Charnes', 'Caccia', 'Bogado', 'San Martin', 'Maradona',
            'Mendez', 'Donado', 'Vila', 'Ancon', 'Mozart', 'Silva', 'Gutierrez', 'Clarke', '1816', '27 De Abril', 'Roca', 'Ercilia', 'Savin', 'Primero', 'Dorado',
            'Fragata', '1373', 'Argentina', '1822', 'Escalada', 'De Rosas', 'De Pineda', 'Pagano', 'Platini', '170', 'Vanzo', 'Colibri', 'Sauna', 'Aborigenes',
            'Humberto', 'Maistegni', 'Las Vias', '100', 'Miranda', '1404', 'Micheli', 'Zola', 'Rosas', 'De La Boca', 'Virasoro', 'Paso', 'Travesía', 'Colon', 
            'Arteaga', 'Salles', 'De La Libertad', 'Janssen', 'Lugones', 'Coaluez', 'Casella', 'Intendente Marcellino', 'Garcia', '1380', '1447', 'Arias', 'Goyena', 
            'Dominguez', 'Cuanaste', 'Israel', 'Illia', 'Giaconne', '1399', 'Casas', 'Carcano', 'Morrison', 'Vigil', 'Mazzaglia', 'Argentino', 'De La Cosa', 'Coria', 
            'Traful', 'Intendente Macilla', 'Boero', 'Solis', '1734', 'Daract', 'Libano', 'Central', 'Galvez', 'Las Vías', 'Muller', 'Pineda', 'Suarez', 'Blanco', 
            '1114', 'Malvinas', 'Margis', 'Vertiz', 'Morales', 'Newbery', 'Aquino']

Por otro lado, hay calles para las cuales el algoritmo de matching cometió errores al asignarles un valor incorrecto. Por ejemplo, 'F More' no debería haber sido asignado a otra calle que no sea 'Felipe Moré'. Además, existen muchos casos en los cuales el nombre de una calle ha cambiado desde el año 2012. 

Estos casos, y muchos más, serán tratados específicamente en el diccionario *'cambios_rosario'*.

In [ ]:
cambios_rosario =  {'Schwesttror': 'Albert Schweitzer', 
                    'Uruburu': 'Avenida José Uriburu',
                    'Urquiza Al 7400': 'Urquiza',
                    ', Roca': 'Roca',
		            'Juan Del Valle' : 'De La Salle',
                    'Sta Rosa': 'José María Rosa',
                    '27': '27 de Febrero',
                    'Juto': 'Juan B. Justo',
                    'Blanca': 'Bahía Blanca',
                    'Linces Furles': 'Pedro Lino Funes',
                    'El Salvador': 'República del Salvador',
                    '25 De Diciembre': 'Juan Manuel de Rosas', # cambió de nombre
                    'Virgilio': 'José Margis', # cambió de nombre
                    'Jb Justo': 'Juan B. Justo',
                    '1380': 'Calle 1880',
                    'Ing Thedy': 'Thedy',
                    'Ruiz Dr Fernando': 'Ruiz',
                    'Juez Zuviria': 'Zuviria',
                    'Valvo': 'Lovalvo',
                    'Jaske': 'Jaques',
                    'Bernhein': 'Bernheim',
                    'Juan Xxiii': 'Juan Pablo XXIII',
                    'Pedro Lino': 'Pedro Lino Funes',
                    'Espa?': 'España',
                    'Dr Pe?': 'David Peña',
                    'Rotonda Venesia': 'Venesia',
                    'Schmild': 'Schmidl Ulrico',
                    '1349': 'Alma Maritano',
                    'Onesimo Leguizamon': 'Leguizamón',
                    'Aldrey': 'Buchanan',  # cambió de nombre
                    '1426': 'Aduana',
                    'Casacuberta': 'Casa Cuberta',
                    'Lola Membrives': 'Membribes',
                    'Cinco De Agosto': '5 de Agosto',
                    'Domingo Silva': 'Silva',
                    'Coronel Arnold': 'Arnold',
                    'Passo': 'Juan José Paso',
                    'Araoz De La Madrid': 'Aráoz',
                    'Rondeu': 'Rondeau',
                    'Bayo': 'Servando Bayo',
                    'Cerrito (2100)': 'Cerrito',
                    'Berruti': 'Luis Beruti',
                    'Berutti': 'Luis Beruti',
                    'Ee Uu': 'Avenida 25 De Mayo',
                    'Mazza': 'Juan Agustín Maza',
                    'Ovidio Andrades': 'Andrade',
                    'Gazcon':'Gascón',
                    'Barro': 'Barra',
                    'Dovado': 'José Agustín Donado', 
                    'Colectora Avenida 25 De Mayo': 'Avenida 25 de Mayo',
                    'Colectora 25 De Mayo':'Avenida 25 de Mayo',
                    'Colombles': 'José Colombres', 
                    'Colombes': 'Avenida Doctor Carlos Colombres', 
                    'Romero Pinedo': 'Avenida Romero de Pineda',
                    'Pizurno': 'Pablo Pizzurno', 
                    'Celedonio Escalada': 'Tunel Escalada', 
                    'Presidente Filiol': 'Fillol', 
                    'Pasaje Rosas': 'Pascual Rosas', 
                    'Eva Avenida Presidente Perón': 'Avenida Eva Perón', 
                    'Acceso Sur': 'General Manuel Belgrano', 
                    'Carduba': 'Eva Perón', 
                    'Bianco Fioni': 'Doctor Rafael Biancofiori', 
                    'De Lucas': 'Esteban de Luca', 
                    'Noris': 'Juan Díaz de Solís', 
                    'Luis Vila': 'Doctor Vila', 
                    'Paulon': 'Pavlov', 
                    'Granadero Baigorria': 'Baigorria', 
                    'Carrilego': 'Carriego', 
                    'Pladon': 'Platón', 
                    'Pasaje Hutchincon': 'Hutchinson', 
                    'Unidas': 'Avenida Provincias Unidas', 
                    'Milton': 'Dellarole', 
                    'Pedro Tuella': 'Tuella', 
                    'Melian': 'José Melian', 
                    'Avalez': 'Benito Álvarez', 
                    'Bering': 'Pasaje Behring', 
                    'Paico': 'Pasco', 
                    'Provincias': 'Avenida Provincias Unidas',
                    'Avenida Corrientes': 'Corrientes', 
                    'Jaureche': 'Arturo Jauretche', 
	                'More': 'Felipe Moré',
                    'Cerecito': 'Cereseto', 
                    'Uno': 'Calle 1', 
                    'Befer': 'Becquer', 
                    'Guillermo Tell': 'Calle 529', 
                    'Conscripto Carrasco': 'Carrasco',
                    'Torres': 'Torre Revello', 
                    'Colectora 25 De Mayo': 'Avenida 25 de Mayo',
                    'Curie': 'Pasaje Madame Curie',
                    'Legarza': 'Pje. Lejarza', 
                    'Juan Pablo Segundo': 'Juan Pablo II', 
                    'Nicolas Laguna': 'Laguna', 
                    'Avenida Brassey': 'Brassey', 
                    'Lagos': 'Ovidio Lagos', 
                    'Ovidiolagos': 'Ovidio Lagos',
                    'Cachi Bis': 'Cachi', 
                    'Pedro Tuella': 'Tuella', 
                    'Colectora 27 De Febrero': '27 De Febrero', 
                    'Adrian': 'Pasaje 1351', 
                    '8 De Noviembre': 'Ocho de Noviembre', 
                    'Humberti 1°': 'Humberto Primero', 
                    'Humberto I': 'Humberto Primero',
                    'Franchini': 'Pasaje Franchini',  
                    'Avenida De Los Inmigrantes': 'Inmigrantes', 
                    'Avenida C Casado': 'Carlos Casado', 
                    'Republica De Argelia': 'Argelia', 
                    'Linneo': 'Pasaje Linneo', 
                    'Zola': 'Emilio Zola', 
                    'Camino Ruiz': 'Doctor Fernando Ruiz', 
                    'Cristal': 'Pasaje 2136', 
                    'Bogado': 'Coronel Félix Bogado', 
                    'Cesar': 'Avenida Gobernador Caesar', 
                    'Cavassa': 'Avenida Cabassa', 
                    'Nicolas Julian': 'Julián Nicolás', 
                    'Firri': 'Doctor Rafael Biancofiori', 
                    'Dr Cue': 'Victor Cue', 
                    'Biselli': 'Sanguinetti', 
                    'De Escalada': 'Remedios de Escalada', 
                    'Dr Casal': 'Mauricio Casal', 
                    'Zabala': 'Zavalla', 
                    'J Casals': 'Joaquina de Casals', 
                    'Cerrillos': 'Pasaje Cemillos', 
                    'Mougfeld': 'Avenida Arquitecto Mongsfeld', 
                    'Cortada Vigil': 'Constancio C. Vigil', 
                    'Ing Venesia': 'Gualberto Venesia', 
                    'Flamarion': 'Camilo Flammarion', 
                    'Maquinista Gallini': 'Gallini', 
                    'Falcon': 'Biblioteca Vigil', 
                    'Tunel': 'Pasaje Túnel', 
                    'Newery': 'Newbery',
                    'Solezzi': 'Zolessi', 
                    'Mandara': 'Pasaje Mansilla', 
                    'Ardeopa': 'Alfredo de Arteaga', 
                    '2126': 'Avellaneda', 
                    'Mu?iz': 'Muñiz', 
                    'Aldrey': 'Buchanan', 
                    'Correo Argentino': 'Central Argentino', 
                    'De Nito': 'Pasaje De Nito', 
                    'Colectora Oeste': 'Colectora de Autopista (Oeste)', 
                    'Campanita': 'Pasaje 2138', 
		            'Int Morcillo': 'Avenida Intendente Morcillo', 
                    'Casacuberta': 'Juan A. Casa Cuberta', 
                    'Manuel Flores': 'Las Flores', 
                    'Boero': 'Pasaje Boero', 
                    'Francisco Tarragona': 'Tarragona', 
                    'Santa Maria De Nicasio Nicasio Oroño': 'Santa María de Oro', 
                    'Opicci': 'Pedro Oppici',  
                    'Juan Pablo Ii (colect Avenida Circunvalacion)': 'Juan Pablo II', 
                    'Dr Sabattini': 'Avenida Amadeo Sabattini', 
                    'Jaske': 'Jaques', 
                    'Aguetta': 'Teniente Agneta', 
                    '1849': 'Calle 1840', 
                    'Rp N&#186; 21': 'Ruta Provincial 21', 
                    '1349': 'Calle 1359',
                    '1331': 'Camino 1331',
                    'Marquez De Moya': 'Moya', 
                    '1404': 'Pasaje 1474', 
                    'Silva 1000': 'Cayetano Silva', 
                    'Monsgfeld': 'Avenida Arquitecto Mongsfeld',
                    'Frondizzi': 'Avenida Arturo Frondizi', 
                    'Hc Agneta': 'Teniente Agneta', 
                    'Ortiz De Guinea': 'Siripo', 
                    '1428': 'Avenida Alberto J. Paz',  
                    'Schmild': 'Schmidl Ulrico', 
                    '1366': 'Calle 1367',  
                    '1507': 'Pasaje 1517', 
                    '1426': 'Aduana',  
                    'Circunvacion': 'Juan Pablo II', 
                    'Sanchez Granel': 'Granel',
                    'Ing Lamas': 'Intendente Luis Lamas',
                    'Cervantes Saavedra': 'Cervantes',
                    'Gob Caesar': 'Avenida Gobernador Caesar',
                    'Tte Sanchez': 'Teniente General Sánchez'}

Una vez definido el diccionario con los cambios, se procede a corregir las calles que el algoritmo de matching confundió. Para ello, se comparan las claves del diccionario *'similares'*, y si alguna coincide con alguna clave del diccionario *'cambios_rosario'*, su valor será reemplazado.

Finalmente, los valores originales del dataframe serán reemplazados según el diccioanario final *'similares'*.

In [ ]:
copia_similares = similares.copy()

for key in copia_similares: # elimino las claves del diccionario 'similares' que figuren en la lista 'eliminar'
    if key in eliminar:
        del similares[key]
    elif key in cambios_rosario: # si la clave coincide con alguna de los cambios, modifico su valor.
        similares[key] = cambios_rosario[key]

# itero sobre el dataframe 'accidentes_ciudad' y actualizo 'calle_lista' según los valores en 'similares'
for index, row in accidentes_ciudad.iterrows():
    for i, calle in enumerate(row['calle_lista']):
        cambio_key = similares.get(calle)
        if cambio_key is not None:
            if isinstance(cambio_key, list):
                accidentes_ciudad.at[index, 'calle_lista'][i] = ' '.join(cambio_key)
            else:
                accidentes_ciudad.at[index, 'calle_lista'][i] = cambio_key

El siguiente bloque de código aborda casos particulares que no pudieron ser tratados con los procedimientos anteriores.

In [ ]:
accidentes_ciudad.loc[14332, 'calle_lista'][accidentes_ciudad.loc[14332, 'calle_lista'].index('')] = 'Pasaje B'
accidentes_ciudad.loc[11446, 'calle_lista'][accidentes_ciudad.loc[11446, 'calle_lista'].index('')] = 'Pasaje A'
accidentes_ciudad.at[87147, 'calle_lista'] = ['Avenida Battle y Ordoñez', 'Calle 2106']
accidentes_ciudad.at[93344, 'calle_lista'] = ['5 de Agosto', 'Calle 521']
accidentes_ciudad.at[393, 'calle_lista'] = ['San Martin', 'Avenida Battle y Ordoñez']
accidentes_ciudad.at[19717, 'calle_lista'] = ['Lopez', 'Silos Davis']
accidentes_ciudad.at[5313, 'calle_lista'] = ['Francia', 'Estado de Israel']
accidentes_ciudad.at[3127, 'calle_lista'] = ['25 de Mayo', 'Sorrento']
accidentes_ciudad.at[3136, 'calle_lista'] = ['25 de Mayo', 'Sorrento']

indices_a_actualizar = [2931, 2947, 2993, 3012, 3026, 3028, 3028, 3030, 5628, 5669, 5673, 8329]

for indice in indices_a_actualizar:
    accidentes_ciudad.at[indice, 'calle_lista'] = [
        'Avenida Battle y Ordoñez' if elem == '' else elem for elem in accidentes_ciudad.at[indice, 'calle_lista']]

Finalmente, se vuelve a calcular el número total de elementos (calles participantes del siniestro) de cada lista de la columna *'calle_lista'*, actualizando los valores de *'numero_calles'*.

In [ ]:
accidentes_ciudad['numero_calles'] = accidentes_ciudad['calle_lista'].apply(lambda x: len(x) if (isinstance(x, list) and len(x) > 0 and x[0].strip() != '') else 0)
counts = accidentes_ciudad['numero_calles'].value_counts()
counts

numero_calles
2    26347
1     4418
0     3156
3       13
Name: count, dtype: int64

## Geocodificar direcciones puntuales con Nominatim

En esta sección se geocodificarán las direcciones puntuales, como Pasco 1550, con Nominatim. En primer lugar, filtraremos los siniestros a geocodificar, es decir, todos ellos que posean valores nulos en la columna *'forma_geocod'*.

In [ ]:
siniestros_a_geocodificar = accidentes_ciudad[accidentes_ciudad['forma_geocod'].isnull()]  #nos quedamos con los que no están geocodificados

Luego, se define la función **geocodificar_dir_puntual**, que recibe una dirección y utiliza Nominatim para obtener su localización geográfica. 

Para evitar errores de geocodificación debido a problemas de conexión o saturación del servidor, se introduce una espera de 3 segundos entre cada iteración y una espera adicional de 2 segundos en caso de que sea necesario reintentar la geocodificación. Este criterio se adoptó después de varios intentos fallidos, donde la saturación del servidor resultaba en geocodificaciones erróneas o en errores que impedían completar la geocodificación de la dirección.

In [ ]:
def geocodificar_dir_puntual(direccion, max_retries=3):
    '''Geocodifica una dirección puntual con Nominatim'''
    geolocator = Nominatim(user_agent="mi-aplicacion", timeout=3)

    if isinstance(direccion, list):
        direccion = direccion[0]

    direccion_completa = f"{direccion}, Rosario, Rosario, Santa Fe, Argentina"
    
    retries = 0
    while retries < max_retries:
        try:
            location = geolocator.geocode(direccion_completa)
            if location:
                latitud = str(location.latitude)
                longitud = str(location.longitude)
                return f"{latitud}, {longitud}", direccion
            else:
                return "no funciono", [direccion]

        except Exception as e:
            print(f"Error en la geolocalización para la dirección {direccion}: {e}")
            retries += 1
            time.sleep(2)  # esperar 2 segundos antes de reintentar, sino a veces da error

    return "no funciono", direccion

La función definida anteriormente se utiliza en el siguiente código, donde se filtran los siniestros con una única calle involucrada. Los resultados se almacenan en las columnas *'forma_geocod'* y *'calles_osm'*.

In [ ]:
forma_geocod_results = []
calles_osm_results = []

# itero sobre las filas del DataFrame original filtrando por 'numero_calles' == 1
for index, row in siniestros_a_geocodificar[siniestros_a_geocodificar['numero_calles'] == 1].iterrows():
    forma_geocod, calles_osm = geocodificar_dir_puntual(row['calle_lista'])
    forma_geocod_results.append(forma_geocod)
    calles_osm_results.append(calles_osm)

calles_osm_results_flattened = [item[0] if isinstance(item, list) else item for item in calles_osm_results]

# Asignar los resultados a nuevas columnas del DataFrame original solo para las filas filtradas
siniestros_a_geocodificar.loc[siniestros_a_geocodificar['numero_calles'] == 1, 'forma_geocod'] = forma_geocod_results
siniestros_a_geocodificar.loc[siniestros_a_geocodificar['numero_calles'] == 1, 'calles_osm'] = calles_osm_results_flattened

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2976\1376719921.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2976\1376719921.py:14: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Mendoza 3900', 'Francia 4600', 'Lamadrid 3225', 'Avellaneda 4900', 'Biedma 3800', 'Sanchez De Loria 1300 Bis', 'Ovidio Lagos 7000', 'Ovidio Lagos 7700', 'Pasaje 1819', 'H Quintana - Francia 5400', 'Cullen 3597', 'Galvez 5239', 'Juan José Paso 8400', 'Eva Peron 8700', 'Francia 3500', 'Bulevar Juan Francisco Seguí 2000', 'Arijon 1200', 'San Martin 6100', 'Entre Rios 2400', 'San Martin 4700', 'Riobamba 200', 'Maipu 1700', 'Galvez 300', 'Frias 481', '

## Geocodificar intersecciones con OSM

La celda de código que se presenta a continuación tiene como objetivo geocodificar intersecciones de calles donde ocurrieron los siniestros. Para ello, se definen dos funciones: **'buscar_alternativas'** y **'geocodificar_intersecciones'**.

Se comienza llamando a la función principal, **'geocodificar_intersecciones'**, la cual recibe una lista de calles (por ejemplo, ['27 de Febrero', 'Francia']), el nombre de un municipio (por ejemplo, 'Rosario'), y una lista de calles oficiales reconocidas por OpenStreetMap (OSM) en ese municipio.
Es importante que el municipio ingresado coincida con uno de los elementos contenidos en la lista *'municipio'*, definida en la sección de limpieza de datos.

Esta función busca encontrar la posición geográfica, si existe, de las calles en la lista proporcionada. Comienza buscando una coincidencia exacta de los nombres en la base de datos de OSM. Si no se encuentra una coincidencia exacta, puede suceder que el nombre esté incompleto (por ejemplo, 'Francia' figura como 'Avenida Francia') o que la calle no exista en la base de datos. En éste ultimo caso, la función devuelve un resultado indicando la situación, como *'calle ppal no encontrada'*, *'calle sec no encontrada'* o *'ambas calles no encontradas'*.

Si la búsqueda inicial no produce resultados, la función llama a otra función interna, **'buscar_alternativas_y_geocodificar'**, que busca alternativas para las calles utilizando la función **buscar_alternativas**. Esta última función devuelve todas las opciones de calles encontradas en la base de datos de OSM que coinciden con la calle ingresada, sin tener en cuenta acentos ni distinción entre mayúsculas y minúsculas. Por ejemplo, para la calle 'Carrasco', devolverá las siguientes alternativas: 'Gabriel Carrasco', 'Avenida Eudoro Carrasco', 'Carrasco', 'Colectora Eudoro Carrasco', 'Omar Carrasco'. Además, en el caso de que se trate de una Colectora o Circunvalación, se agregan otras opciones válidas a la lista de calles alternativas.

Una vez que se han encontrado todas las posibles calles, la función principal busca la intersección entre ellas. Puede devolver los siguientes resultados:

- 'ppal calles no encontrada': No se encontró la calle principal ni ninguna coincidencia con la misma.
- 'sec calles no encontrada': No se encontró la calle secundaria ni ninguna coincidencia con la misma.
- 'ambas calles no encontradas': No se encontraron ambas calles ni ninguna coincidencia con las mismas.
- 'chequear': La intersección devolvió una colección geométrica. 
- 'error': No se halló la intersección.


In [ ]:
def buscar_alternativas(calle, calles_osm_verif):
    '''Busca calles alternativas en la base de datos de OSM para la calle recibida'''
    calle = unidecode(calle).lower() # convierto la calle sin acentos y a minúsculas
    alternativas = [c for c in calles_osm_verif if unidecode(c.lower()).find(calle) != -1]
    palabras_colectora = ['circunvalacion', 'circunvalación', '25 de mayo', 'colectora'] 
    alternativas_colectora = ['Juan Pablo II', 'Colectora Juan Pablo II', 'Colectora José María Rosa', 'José María Rosa']
    
    # si la calle contiene alguna de las palabras de colectora, entonces agrego calles adicionales
    if any(palabra in calle for palabra in palabras_colectora):
        alternativas.extend(alternativas_colectora)

    return alternativas

def geocodificar_intersecciones(calles, municipio, calles_osm_verif):
    '''Geocodifica intersecciones según el municipio y la lista de calles pasadas por parámetro'''
    calle_ppal = unidecode(calles[0]).lower()
    calle_sec = unidecode(calles[1]).lower()

    if not calle_ppal.strip() or not calle_sec.strip():
        return 'error', 'error'

    # columna de nombres de calles sin acentos y en minúsculas
    calles_bd_sin_acento = datos_municipios[municipio]['edges']['name'].apply(lambda x: unidecode(x).lower())
    
    # filas que contienen las calles buscadas
    mask_calle_ppal = calles_bd_sin_acento.str.contains(calle_ppal, case=False, regex=False)
    mask_calle_sec = calles_bd_sin_acento.str.contains(calle_sec, case=False, regex=False)
    
    calle_ppal_geo = datos_municipios[municipio]['edges'][mask_calle_ppal]
    calle_sec_geo = datos_municipios[municipio]['edges'][mask_calle_sec]
    
    # nombres exactos
    nombre_exacto_ppal = datos_municipios[municipio]['edges'][mask_calle_ppal]['name'].unique().tolist()
    nombre_exacto_sec = datos_municipios[municipio]['edges'][mask_calle_sec]['name'].unique().tolist()
    
    # si no reconoce el nombre de una o ambas calles
    if not nombre_exacto_ppal or not nombre_exacto_sec:
        flag = 'ambas'
        if not nombre_exacto_ppal and nombre_exacto_sec:
            flag = 'ppal'
        elif nombre_exacto_ppal and not nombre_exacto_sec:
            flag = 'sec'
        return "{} calles no encontradas".format(flag), 'error'
    
    calles_osm = [nombre_exacto_ppal[0], nombre_exacto_sec[0]]
    
    calle_ppal_geo = calle_ppal_geo.unary_union
    calle_sec_geo = calle_sec_geo.unary_union
    
    # función para buscar alternativas de ambas calles si la intersección no devuelve un punto
    def buscar_alternativas_y_geocodificar(calle_ppal, calle_sec, calles_osm_verif):
        alternativas_ppal = buscar_alternativas(calle_ppal, calles_osm_verif)
        alternativas_sec = buscar_alternativas(calle_sec, calles_osm_verif)
        calles_con_interseccion = []
        
        for alt_ppal in alternativas_ppal:
            for alt_sec in alternativas_sec:
                calle_geo_ppal = datos_municipios[municipio]['edges'][datos_municipios[municipio]['edges']['name'].str.contains(alt_ppal, case=False, regex=False)]
                calle_geo_sec = datos_municipios[municipio]['edges'][datos_municipios[municipio]['edges']['name'].str.contains(alt_sec, case=False, regex=False)]
                
                if len(calle_geo_ppal) > 0 and len(calle_geo_sec) > 0:
                    calle_geo_ppal = calle_geo_ppal.unary_union
                    calle_geo_sec = calle_geo_sec.unary_union
                    
                    inter = calle_geo_ppal.intersection(calle_geo_sec)
                    
                    if isinstance(inter, Point):
                        calles_con_interseccion.append((alt_ppal, alt_sec))
                        return inter, calles_con_interseccion
                    if isinstance(inter, MultiPoint):
                        if inter.geoms[0].equals_exact(inter.geoms[1], tolerance=0.0001):
                            calles_con_interseccion.append((alt_ppal, alt_sec))
                            return inter.centroid, calles_con_interseccion
                        elif inter.geoms[0].equals_exact(inter.geoms[1], tolerance=0.001):
                            calles_con_interseccion.append((alt_ppal, alt_sec))
                            return inter.centroid, calles_con_interseccion
                        
    try:
        inter = calle_ppal_geo.intersection(calle_sec_geo)
        
        if isinstance(inter, MultiPoint):
            if inter.geoms[0].equals_exact(inter.geoms[1], tolerance=0.0001):
                return inter.centroid, calles_osm
            elif inter.geoms[0].equals_exact(inter.geoms[1], tolerance=0.001):
                return inter.centroid, calles_osm
            else:
                centroide, calles_lista = buscar_alternativas_y_geocodificar(calles[0], calles[1], calles_osm_verif)
                if calles_lista is not None:
                    return centroide, calles_lista
                    #return "chequear puntos distantes", 'error'
        
        if isinstance(inter, GeometryCollection):
            return "chequear", 'error'
        
        if isinstance(inter, LineString) or isinstance(inter, MultiLineString):
            centroide, calles_lista = buscar_alternativas_y_geocodificar(calles[0], calles[1], calles_osm_verif)
            if calles_lista is not None:
                return centroide, calles_lista
        return inter, calles_osm
    except Exception as e:
        return 'error', 'error'

En el siguiente código, se llama de forma iterativa a la función previamente definida, **'geocodificar_intersecciones'**, y se almacenan sus resultados en las listas definidas al comienzo. Luego, se asignan esos valores a las columnas *'forma_geocod'* y *'calles_osm'* del dataframe. Esto se aplica únicamente a las filas donde el número de calles que intervienen en el siniestro es igual a 2.

In [ ]:
forma_geocod_results = []
calles_osm_results = []

for index, row in siniestros_a_geocodificar[siniestros_a_geocodificar['numero_calles'] == 2].iterrows():
    forma_geocod, calles_osm = geocodificar_intersecciones(row['calle_lista'], 'Rosario', calles_osm_rosario)
    forma_geocod_results.append(str(forma_geocod)) 

    # convierto cada lista de calles a un string con cada calle separada por comas, pero mantengo 'error'
    calles_osm_str = '; '.join(map(str, calles_osm)) if not 'error' in calles_osm else 'error'
    calles_osm_results.append(calles_osm_str)

siniestros_a_geocodificar.loc[siniestros_a_geocodificar['numero_calles'] == 2, 'forma_geocod'] = forma_geocod_results
siniestros_a_geocodificar.loc[siniestros_a_geocodificar['numero_calles'] == 2, 'calles_osm'] = calles_osm_results

A continuación, se procede a filtrar todos los siniestros que ya se encontraban geocodificados, es decir, aquellos en los que la columna *'forma_geocod'* ya contenía información, y se concatenan estos siniestros filtrados con el dataframe resultante 'accidentes_ciudad'.

In [ ]:
registros_con_geocod = accidentes_ciudad[accidentes_ciudad['forma_geocod'].notnull()] # filtro filas donde 'forma_geocod' no es nulo
siniestros_rosario_final = pd.concat([siniestros_a_geocodificar, registros_con_geocod])  # concateno ambos df's
siniestros_rosario_final = siniestros_rosario_final.reset_index(drop=True)

Para extraer las latitudes y longitudes de cada coordenada, se define la función **'extraer_coordenadas'**. Esta función se aplica a todas las filas que contienen información en la columna *'forma_geocod'*. Los resultados se asignan a dos nuevas columnas: *'lat'* para las latitudes y *'lon'* para las longitudes.

In [ ]:
def extraer_coordenadas(geo):
    '''Extrae la latitud y longitud de las coordenadas pasadas por parámetro'''
    if isinstance(geo, str):
        if geo in ['error', 'no funciono', 'ppal calles no encontradas', 'sec calles no encontradas', 'ambas calles no encontradas', 'chequear', 'chequear puntos distantes']:
            return None, None
        try:
            lat, lon = map(float, geo.split(','))
            return lat, lon
        except ValueError:
            if 'POINT' in geo:
                geo_cleaned = geo.replace('POINT', '').replace('(', '').replace(')', '')
                coords = geo_cleaned.split()
                return float(coords[1]), float(coords[0])  # revertir el orden de latitud y longitud
    return None, None

# filtro las filas donde 'lat' y 'lon' son nulos y la columna 'forma_geocod' ya tenga info
mask = (~siniestros_rosario_final['forma_geocod'].isnull()) & (~siniestros_rosario_final['forma_geocod'].isin(['error', 'no funciono', 'chequear puntos distantes', 'chequear', 'ppal calles no encontradas', 'sec calles no encontradas', 'ambas calles no encontradas']))
coordenadas = siniestros_rosario_final.loc[mask, 'forma_geocod'].apply(lambda x: extraer_coordenadas(x))
siniestros_rosario_final.loc[mask, ['lat', 'lon']] = coordenadas.tolist()

Finalmente, se exporta el dataframe finalizado a un csv con el mismo nombre.

In [ ]:
siniestros_rosario_final.to_csv("siniestros_geocod_rosario2.csv")